First, install and import all neccessary python libraries.

In [21]:
!pip install gensim
!pip install keras
!pip install nltk
!pip install numpy
!pip install pandas
!pip install sklearn
!pip install tensorflow==1.15
!pip install wordcloud

import gensim
from gensim.models import word2vec
from itertools import cycle
import json
from keras.utils import np_utils
import logging
import matplotlib.pyplot as plt
import nltk
import numpy as np
import os
import pandas as pd
import re
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KDTree
from sklearn.preprocessing import LabelEncoder
import sys
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Bidirectional, Dense, Embedding, SimpleRNN, LSTM
from wordcloud import WordCloud, ImageColorGenerator

nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/lindawong/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/lindawong/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /Users/lindawong/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /Users/lindawong/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /Users/lindawong/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /Users/lindawong/nltk_data...
[nltk_data]    |   Package cess_cat is already up-

True

In [22]:
dataset_folder = input("Enter path of the Yelp dataset folder: ")

Enter path of the Yelp dataset folder: ./yelp_dataset


From the original Yelp dataset, we will extract data from review.json and business.json.

In [23]:
# functions for data extraction
def extract_data(business_data_folder, review_data_folder):
    output_data = {}
    with open(business_data_folder) as bd_handle:
        business = bd_handle.readline()

        while business:
            bd_data = json.loads(business)

            if bd_data['categories']:
                output_data[bd_data['business_id']] = {
                    'categories': [cat.strip() for cat in bd_data['categories'].split(',')],
                    'reviews': []
                }

            business = bd_handle.readline()

        bd_handle.close()

    with open(review_data_folder) as rd_handle:
        review = rd_handle.readline()

        while review:
            rv_data = json.loads(review)

            if rv_data['business_id'] in output_data:
                review = {}
                review['review_id'] = rv_data['review_id']
                review['text'] = rv_data['text']
                review['useful'] = rv_data['useful']
                review['funny'] = rv_data['funny']
                review['cool'] = rv_data['cool']
                output_data[rv_data['business_id']]['reviews'].append(review)

            review = rd_handle.readline()

        bd_handle.close()
        
    return output_data

def write_extracted_data(output_data, output_file):
    data_to_write = []
    for business_id in output_data:
        categories = output_data[business_id]['categories']

        if 'Restaurants' in categories and len(output_data[business_id]['reviews']) > 0:
            cat_to_write = ':'.join(categories)
            for review in output_data[business_id]['reviews']:
                review_id = review['review_id']
                text = review['text']
                useful = str(review['useful'])
                funny = str(review['funny'])
                cool = str(review['cool'])
                data_to_write.append({
                    'business_id': business_id,
                    'categories': cat_to_write,
                    'review_id': review_id,
                    'text': text,
                    'useful': useful,
                    'funny': funny,
                    'cool': cool
                })           

    with open(output_file, 'w') as of_handle:
        json.dump(data_to_write, of_handle)
        of_handle.close()

In [24]:
business_data_folder = os.path.join(dataset_folder, 'business.json')
review_data_folder = os.path.join(dataset_folder, 'review.json')
output_data = extract_data(business_data_folder, review_data_folder)
dataset_file = './dataset.json'
write_extracted_data(output_data, dataset_file)

Now, the data we need from review.json and business.json are stored in the file "dateset.json". Next, we will clean up the data and write the preprocessed datasets to "normalised_data.json".

!!!TODO: explain clipping, making sure dataset has even ranges of labels

In [25]:
# functions for data preprocessing
def preprocess_data(data_file):

    df = pd.read_json(data_file)
    df['useful'] = df['useful'].astype(float)
    df['funny'] = df['funny'].astype(float)
    df['cool'] = df['cool'].astype(float)
    print("Dataset size: {0:d} rows x {1:d} columns".format(df.shape[0], df.shape[1]))

    vote_labels = ['useful', 'funny', 'cool']
    useful_min, useful_max = df['useful'].min(), df['useful'].max()
    funny_min, funny_max = df['funny'].min(), df['funny'].max()
    cool_min, cool_max = df['cool'].min(), df['cool'].max()

    # print out distribution of reviews by their vote counts
    for i in range(len(vote_labels)):
        ranges = np.arange(-1, 100, 10)
        counts = df.groupby(pd.cut(df[vote_labels[i]], ranges)).count()[vote_labels[i]]
        print(counts)
        print("Out of range counts: ", df.shape[0]-counts.sum())

    # save raw vote counts
    df['useful_raw'] = df['useful']
    df['funny_raw'] = df['funny']
    df['cool_raw'] = df['cool']
    # clip vote counts to specific ranges
    df['useful'] = df['useful'].clip(3.0, 27.0)
    df['funny'] = df['funny'].clip(3.0, 20.0)
    df['cool'] = df['cool'].clip(3.0, 24.0)

    # normalize data with min-max scaling
    min_max_scaler = preprocessing.MinMaxScaler()
    useful_scaled = min_max_scaler.fit_transform(np.array(df['useful']).reshape(-1,1))
    df['useful'] = pd.DataFrame(useful_scaled)
    funny_scaled = min_max_scaler.fit_transform(np.array(df['funny']).reshape(-1,1))
    df['funny'] = pd.DataFrame(funny_scaled)
    cool_scaled = min_max_scaler.fit_transform(np.array(df['cool']).reshape(-1,1))
    df['cool'] = pd.DataFrame(cool_scaled)

    ranges = [-0.1, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    processed_data = []

    # balance dataset for each of vote category
    # make sure there is an even number of reviews in each increment of 0.1
    # from range 0.0 to 1.0
    for i in range(len(vote_labels)):
        counts = df.groupby(pd.cut(df[vote_labels[i]], ranges)).count()[vote_labels[i]]
        min_count = counts.min()
        new_df = pd.DataFrame()
        for j in range(10):
            lower = j/10
            upper = (j+1)/10 if j < 9 else 1.1
            new_data = df[(df[vote_labels[i]] >= lower) & (df[vote_labels[i]] < upper)]
            new_data = new_data.sample(frac=1)
            new_data = new_data.head(min_count)
            new_df = new_df.append(new_data)

        processed_data.append(new_df)
    
    return processed_data


def write_extracted_data(useful, funny, cool):
    output_dict = {
        'useful': useful.to_json(orient = 'records'),
        'funny': funny.to_json(orient = 'records'),
        'cool': cool.to_json(orient = 'records')
    }

    with open('normalised_data.json', 'w') as op_handle:
        json.dump(output_dict, op_handle)
        op_handle.close()

In [15]:
processed_data = preprocess_data(dataset_file)
write_extracted_data(processed_data[0], processed_data[1], processed_data[2])

Dataset size: 4201684 rows x 7 columns
useful
(-1, 9]     4148278
(9, 19]       40402
(19, 29]       7225
(29, 39]       2674
(39, 49]       1197
(49, 59]        623
(59, 69]        415
(69, 79]        245
(79, 89]        153
(89, 99]        119
Name: useful, dtype: int64
Out of range counts:  353
funny
(-1, 9]     4180159
(9, 19]       15647
(19, 29]       3281
(29, 39]       1236
(39, 49]        564
(49, 59]        282
(59, 69]        159
(69, 79]         89
(79, 89]         52
(89, 99]         41
Name: funny, dtype: int64
Out of range counts:  174
cool
(-1, 9]     4173089
(9, 19]       20001
(19, 29]       4550
(29, 39]       1778
(39, 49]        913
(49, 59]        477
(59, 69]        316
(69, 79]        179
(79, 89]        134
(89, 99]         95
Name: cool, dtype: int64
Out of range counts:  152


Create a word2vec model using vocabulary of reviews in "dataset.json".

In [43]:
# functions for creating the word2vec model
def read_data(dataset_file):
    logging.info("Reading data...")
    with open(dataset_file, 'r') as fh:
        full_data = json.load(fh)
        useful_data = pd.DataFrame(json.loads(full_data['useful']))
        funny_data = pd.DataFrame(json.loads(full_data['funny']))
        cool_data = pd.DataFrame(json.loads(full_data['cool']))
        all_data = useful_data.append([funny_data, cool_data])
    
    return useful_data, funny_data, cool_data, all_data


def clean_data(tokenized_data_file, all_data, tokenizer, stop_words):
    logging.info("Cleaning data...")
    with open(tokenized_data_file, 'w') as fh:
        reviews = all_data['text'].tolist()

        for index in range(len(reviews)):
            review = reviews[index]
            no_tabs = str(review).replace('\t', ' ').replace('\n', '')
            alphas_only = re.sub("[^a-zA-Z\.]", " ", no_tabs)
            multi_spaces = re.sub(" +", " ", alphas_only)
            no_spaces = multi_spaces.strip()
            clean_text = no_spaces.lower()
            sentences = tokenizer.tokenize(clean_text)
            sentences = [re.sub("[\.]", "", sentence) for sentence in sentences]

            if len(clean_text) > 0 and clean_text.count(' ') > 0:
                for sentence in sentences:
                    sentence = sentence.split(' ')
                    pruned_sentence = [word for word in sentence if word not in stop_words]
                    sentence = ' '.join(pruned_sentence)
                    if sentence: 
                        fh.write("%s\n" % sentence)

            if (index % 5000) == 0:
                fh.flush()
        fh.close()


def create_wve(tokenized_data_file):
    embedding_size = 250
    min_word_count = 50
    context = 30
    downsampling = 1e-2
    logging.info("Creating word vectors...")
    model = word2vec.Word2Vec(
        word2vec.LineSentence(tokenized_data_file),
        size=embedding_size,
        min_count=min_word_count,
        window=context,
        sample=downsampling)
    model.init_sims(replace=True)
    model.save('./wve.model')
    
    return model

In [44]:
useful_data, funny_data, cool_data, all_data = read_data('normalised_data.json')
tokenized_data_file = './tokennized.txt'
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
stop_words = nltk.corpus.stopwords.words()
clean_data(tokenized_data_file, all_data, tokenizer, stop_words)
wve_model = create_wve(tokenized_data_file)

In [42]:
class BiLSTMModel:
    def __init__(self, data_file, label):
        self.data_file = data_file
        self.label = label
        self.df = []
        self.embedding_dim = 50

    def read_data(self):        
        # read normalized datatsets
        with open(self.data_file, 'r') as f:
            full_data = json.load(f)
            self.df = pd.DataFrame(json.loads(full_data[slef.label]))
    
    def build_model(self):
        for i in range(len(self.df_text)):
            j = 0
            while j < len(self.df_text[idx][i]):
                word = self.df_text[idx][i][j]
                if word in common_words:
                    self.df_text[idx][i].pop(j)
                else:
                    j += 1

        max_length = max([len(s) for s in self.df_text[idx]])
        num_reviews = len(self.df_text[idx])
